In [1]:
from google.colab import drive
import os
if not os.path.exists('/content/drive'): drive.mount('/content/drive')
% cd /content/drive/MyDrive/Dissertation/ad945-diss-project/

Mounted at /content/drive
/content/drive/MyDrive/Dissertation/ad945-diss-project


In [5]:
!pip install sacrebleu
!pip install sentencepiece
!pip install py-rouge

     |████████████████████████████████| 61kB 2.8MB/s 


In [7]:
import torch

import time
import numpy as np
import pickle

import sentencepiece as spm
from tqdm.notebook import tnrange
import sacrebleu
from datetime import datetime

from src.opt import Opt
from src.transformer_layers import Transformer
from src.utils import *
from src.save_load import *
from src.training import *

In [8]:
opt = Opt()

In [ ]:
if not os.path.exists(f"{opt.model_file}{opt.src_lang}.model"):
    move(opt)

In [ ]:
if not os.path.exists(f"{opt.model_file}{opt.src_lang}.model"):
    print('building src processor') 
    ! spm_train --input={opt.input_file}{opt.src_lang} \
    --model_prefix={opt.model_file}{opt.src_lang}.model \
    --vocab_size=8000 --character_coverage=1.0 \
    --model_type=bpe --pad_id=3 --bos_id=-1 \
    --eos_id=-1

    
if not os.path.exists(f"{opt.model_file}{opt.trg_lang}.model"):
    print('building trg processor')
    ! spm_train --input={opt.input_file}{opt.trg_lang} \
    --model_prefix={opt.model_file}{opt.trg_lang}.model \
    --vocab_size=8000 --character_coverage=1.0 \
    --model_type=bpe --pad_id=3

In [ ]:
create_fields(opt)
create_dataset(opt)

In [ ]:
opt.train_len = sum((len(opt.src_bins[v])*v) // opt.tokensize + 1 for v in opt.bins) 
print(opt.train_len)
print(sum(len(opt.src_bins[v]) for v in opt.bins))

In [ ]:
load_dev_dataset(opt)

In [ ]:
model, optim = load_model(opt)

In [ ]:
try:
    opt.k = 10
    opt.print_every = 200 
    opt.save_every = 5000
    opt.epochs = 10
    opt.warmup_steps = 16000
    opt.keep_training = True
    train_model(model, opt)

except Exception as e:
    log.print(e, type=Log.ERROR)
    log.flush()
    raise e